# 解析库的使用

介绍 lxml, Beautiful Soup, pyquery 三种解析库的用法。

## 4.1 XPath

XPath 的几个常用规则：
* nodename &emsp; 选取此节点的所有子节点
* / &emsp; 从当前节点选取直接子节点
* // &emsp; 从当前节点选取子孙节点
* . &emsp; 选取当前节点
* .. &emsp; 选取当前节点的父节点
* @ &emsp; 选取属性

In [3]:
from lxml import etree
text = '''
<div>
<ul>
<li class="item-0"><a href="link1.html">first item</a></li>
<li class="item-1"><a href="link2.html">second item</a></li>
<li class="item-inactive"><a href="link3.html">third item</a></li>
<li class="item-1"><a href="link4.html">fourth item</a></li>
<li class="item-0"><a href="link5.html">fifth item</a></li>
</ul>
</div>
'''
html = etree.HTML(text)  # 也可读取文本文件，html = etree.parse('./text.html', etree.HTMLParser())
result = html.xpath('//*')
print(result)

[<Element html at 0x10f889fc8>, <Element body at 0x10f12f188>, <Element div at 0x10f19a908>, <Element ul at 0x10f244b48>, <Element li at 0x10f244d88>, <Element a at 0x10f244188>, <Element li at 0x10f2448c8>, <Element a at 0x10f908508>, <Element li at 0x10f908548>, <Element a at 0x10f2442c8>, <Element li at 0x10f9085c8>, <Element a at 0x10f908648>, <Element li at 0x10f908588>, <Element a at 0x10f908608>]


这里 * 代表匹配所有节点，也就是所有 HTML 文本中所有节点都会被获取。

**子节点**

如果想获取所有 li 节点：

In [4]:
result = html.xpath('//li')

通过 / 或 // 可查找元素的子节点或子孙节点。假如要选择 li 节点的所有直接 a 子节点：

In [5]:
result = html.xpath('//li/a')

**父节点**

使用 .. 或 parent:: 来获取父节点，比如，现在首先选中 href 属性为 link4.html 的 a 节点，然后再获取其父节点，然后再获取其 class 属性。

In [6]:
result = html.xpath('//a[@href="link4.html"]/../@class')
# 或者
result = html.xpath('//a[@href="link4.html"]/parent::*/@class')

**属性匹配**

可以使用 @ 符号进行属性过滤。比如，如果要选取 class 为 item-0 的 li 节点，可以这样实现：

In [7]:
result = html.xpath('//li[@class="item-0"]')

**文本获取**

用 XPath 中的 text() 方法获取节点中的文本。

In [9]:
result = html.xpath('//li[@class="item-0"]/a/text()')
result

['first item', 'fifth item']

**属性获取**

还是使用 @ 来获取节点属性。例如，想获取所有 li 节点下所有 a 节点的 href 属性：

In [10]:
result = html.xpath('//li/a/@href')
result

['link1.html', 'link2.html', 'link3.html', 'link4.html', 'link5.html']

**属性多值匹配**

有时候，某些节点的某个属性可能有多个值，比如：
```
<li class="li li-first"><a href="link.html">first item</a></li>
```
需要使用 contains() 函数：

In [ ]:
result = html.xpath('//li[@contains(@class, "li")]/a/text()')

**多属性匹配**

根据多个属性确定一个节点，这时就需要同时匹配多个属性。此时可以使用 and 来连接。

In [12]:
from lxml import etree
text = '<li class="li li-first" name="item"><a href="link.html">first item</a></li>'
html = etree.HTML(text)
result = html.xpath('//li[contains(@class, "li") and @name="item"]/a/text()')
result

['first item']

**按序选择**

如果某些属性可能同时匹配了多个节点，可以利用中括号传入索引的方法获取特定次序的节，或者使用 last()、position() 等。

In [13]:
result = html.xpath('//li[1]/a/text()')  # 序号以 1 开头
result = html.xpath('//li[last()]/a/text()')  # 最后一个节点
result = html.xpath('//li[position()<3]/a/text()')  # 位置小于 3 的节点

**节点轴选择**

XPath 提供了很多节点轴选择方法，包括获取子元素、兄弟元素、父元素、祖先元素等。

In [14]:
result = html.xpath('//li[1]/ancestor::*')  # 获取所有祖先节点
result = html.xpath('//li[1]/attribute::*')  # 获取所有属性值
result = html.xpath('//li[1]/child::a[@href="link1.html"]')  # 获取所有直接子节点中 href 属性为 link1.html 的 a 节点
result = html.xpath('//li[1]/descendant::span')  # 获取所有子孙节点中的 span 节点
result = html.xpath('//li[1]/following::*')  # 获取当前节点之后的所有节点

## 4.2 Beautiful Soup

Beautiful Soup 是 Python 的一个 HTML 或 XML 的解析库，可以用它来方便地从网页中提取数据。

Beautiful Soup 在解析时依赖解析器，除了支持 Python 标准库中的 HTML 解析器外，还支持一些第三方解析器（比如 lxml）。
```
BeautifulSoup(markup, 'html.parser')
BeautifulSoup(markup, 'lxml')
```

### 节点选择器

直接调用节点的名称就可以选择节点元素，再调用 string 属性就可以得到节点内的文本。

In [16]:
from bs4 import BeautifulSoup

html = """
<html><head><title>The Dormouse's story</title></head>
<body>
<p class="title" name="dromouse"><b>The Dormouse's story</b></p>
<p class="story">Once upon a time there were three little sisters; and their name were
<a href="http://example.com/elsie" class="sister" id="link1"><!-- Elsie --></a>,
<a href="http://example.com/lacie" class="sister" id="link2">Lacie</a> and
<a href="http://example.com/tillie" class="sister" id="link3">Tillie</a>;
and they lived at the bottom of a well.</p>
<p class="story">...</p>
"""

soup = BeautifulSoup(html, 'lxml')
print(soup.title)
print(soup.title.string)
print(soup.head)
print(soup.p)  # 这种方式只会选择第一个匹配的节点

<title>The Dormouse's story</title>
The Dormouse's story
<head><title>The Dormouse's story</title></head>
<p class="title" name="dromouse"><b>The Dormouse's story</b></p>


**提取信息**

（1）利用 name 属性获取节点的名称。

In [17]:
print(soup.title.name)

title


（2）调用 attrs 获取所有属性。

In [18]:
print(soup.p.attrs)
print(soup.p.attrs['name'])  # 或者 soup.p['name']

{'class': ['title'], 'name': 'dromouse'}
dromouse


（3）利用 string 属性获取节点元素包含的文本内容。

In [19]:
print(soup.p.string)

The Dormouse's story


**嵌套选择**

在上面的例子中，每一个返回结果都是 bs4.element.Tag 类型，它同样可以继续调用节点进行下一步的选择。比如，获取 head 节点元素，可以继续调用 head 来获取其内部的 head 节点元素。

In [20]:
html = """
<html><head><title>The Dormouse's story</title></head>
<body>
"""
soup = BeautifulSoup(html, 'lxml')
print(soup.head.title)
print(type(soup.head.title))
print(soup.head.title.string)

<title>The Dormouse's story</title>
<class 'bs4.element.Tag'>
The Dormouse's story


**关联选择**

在做选择的时候，有时候不能做到一步就选到想要的节点元素，需要先选中某一个节点，然后以它为基准再选择它的子节点、父节点、兄弟节点等。

（1）子节点和子孙节点

选取节点元素之后，如果想要获取它的直接子节点，可以调用 contents 属性，也可以使用 children 属性。如果要得到所有的子孙节点的话，可以调用 descendants 属性。

（2）父节点和祖先节点

如果要获取某个节点元素的父节点，可以调用 parent 属性（获取直接父节点）；如果想获取所有的祖先节点，可以调用 parents 属性。

（3）兄弟节点

使用 next_sibling 和 previous_sibling 分别获取节点的下一个和上一个兄弟元素。

### 方法选择器

上面的方法都是通过属性来选择，不适合进行复杂的选择。Beautiful Soup 还提供了一些查询方法，比如 find_all() 和 find()，通过传入响应的参数，就可以灵活查询。

**find_all()**

查询所有符合条件的元素，传入一些属性或文本，就可以得到符合条件的元素。API 为：
```
find_all(name, attrs, recursive, text, **kargs)
```

（1）通过节点名来查询：

In [23]:
html = """
<div class="panel">
<div class="panel-heading">
<h4>hello</h4>
</div>
<div class="panel-body">
<ul class="list" id="list-1">
<li class="element">Foo</li>
<li class="element">Bar</li>
<li class="element">Jay</li>
</ul>
<ul class="list list-small" id="list-2">
<li class="element">Foo</li>
<li class="element">Bar</li>
</ul>
</div>
</div>
"""
soup = BeautifulSoup(html, 'lxml')
print(soup.find_all(name='ul'))

[<ul class="list" id="list-1">
<li class="element">Foo</li>
<li class="element">Bar</li>
<li class="element">Jay</li>
</ul>, <ul class="list list-small" id="list-2">
<li class="element">Foo</li>
<li class="element">Bar</li>
</ul>]


（2）通过属性来查询：

In [24]:
print(soup.find_all(attrs={'id': 'list-1'}))

[<ul class="list" id="list-1">
<li class="element">Foo</li>
<li class="element">Bar</li>
<li class="element">Jay</li>
</ul>]


对于常用的属性，比如 id 和 class 等，可以直接传入 id 和 class_（为了和 Python 关键字 class 区分）来查询：

In [25]:
print(soup.find_all(id='list-1'))
print(soup.find_all(class_='element'))

[<ul class="list" id="list-1">
<li class="element">Foo</li>
<li class="element">Bar</li>
<li class="element">Jay</li>
</ul>]
[<li class="element">Foo</li>, <li class="element">Bar</li>, <li class="element">Jay</li>, <li class="element">Foo</li>, <li class="element">Bar</li>]


（3）使用 text 参数。text 参数可用来匹配节点的文本，传入的形式可以是字符串，可以是正则表达式对象：

In [26]:
import re
html = '''
<div class="panel">
<div class="panel-body">
<a>Hello, this is a link</a>
<a>Hello, this is a link, too</a>
</div>
</div>
'''
soup = BeautifulSoup(html, 'lxml')
print(soup.find_all(text=re.compile('link')))

['Hello, this is a link', 'Hello, this is a link, too']


**find()**

find() 和 find_all() 的区别是，find() 返回的是单个元素，也就是第一个匹配的元素，而 find_all() 返回的是所有匹配的元素组成的列表。

In [27]:
print(soup.find(text=re.compile('link')))

Hello, this is a link


此外，还有其它查询方法：

* find_parents()、find_parent()：前者返回所有祖先节点，后者返回直接父节点；
* find_next_siblings、find_next_sibling()：前者返回后面所有的兄弟节点，后者返回后面第一个兄弟节点；
* find_previous_siblings()、find_previous_sibling()：前者返回前面所有的兄弟节点，后者返回前面第一个兄弟节点。
* find_all_next()、find_next()：前者返回节点后所有符合条件的节点，后者返回第一个符合条件的节点；
* find_all_previous()、find_previois：前者返回节点前所有符合条件的节点，后者返回第一个符合条件的节点。

### CSS 选择器

使用 CSS 选择器时，只需要调用 select() 方法，传入相应的 CSS 选择器即可。

In [30]:
html = """
<div class="panel">
<div class="panel-heading">
<h4>hello</h4>
</div>
<div class="panel-body">
<ul class="list" id="list-1">
<li class="element">Foo</li>
<li class="element">Bar</li>
<li class="element">Jay</li>
</ul>
<ul class="list list-small" id="list-2">
<li class="element">Foo</li>
<li class="element">Bar</li>
</ul>
</div>
</div>
"""
soup = BeautifulSoup(html, 'lxml')
print(soup.select('.panel .panel-heading'))
print(soup.select('ul li'))
print(soup.select('#list-2 .element'))
print(type(soup.select('ul')[0]))

[<div class="panel-heading">
<h4>hello</h4>
</div>]
[<li class="element">Foo</li>, <li class="element">Bar</li>, <li class="element">Jay</li>, <li class="element">Foo</li>, <li class="element">Bar</li>]
[<li class="element">Foo</li>, <li class="element">Bar</li>]
<class 'bs4.element.Tag'>


**嵌套选择**

select() 方法同样支持嵌套选择。

In [32]:
for ul in soup.select('ul'):
    print(ul.select('li'))

[<li class="element">Foo</li>, <li class="element">Bar</li>, <li class="element">Jay</li>]
[<li class="element">Foo</li>, <li class="element">Bar</li>]


**获取属性**

In [33]:
for ul in soup.select('ul'):
    print(ul['id'])
    print(ul.attrs['id'])

list-1
list-1
list-2
list-2


**获取文本**

要获取文本，可以使用 string 属性，也可以使用 get_text() 方法：

In [34]:
for li in soup.select('li'):
    print('Get Text:', li.get_text())
    print('String:', li.string)

Get Text: Foo
String: Foo
Get Text: Bar
String: Bar
Get Text: Jay
String: Jay
Get Text: Foo
String: Foo
Get Text: Bar
String: Bar
